In [6]:
# Load csv data

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib.pyplot as plt
import spacy 
from spacy.matcher import Matcher
from spacy.scorer import Scorer
from spacy.training import Example
from spacy.tokens import Doc
import pandas as pd
import re
from collections import Counter
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score
import numpy as np
import copy
import csv

filepath = os.path.join(os.getcwd(), 'QueryResults_sample.csv')

stack_posts = pd.read_csv(filepath, sep = ",")

print("loaded csv data")

loaded csv data


In [2]:
# drop all duplicates in posts
df = pd.DataFrame(stack_posts[0:500])
df = df.drop_duplicates(["QuestionId"])
df = df.sort_values(by=["QuestionId"])
#df.duplicated(["QuestionId"])


# get all tags of questions
tag_set = set()
tag_list = []
for tags in df["Tags"]:
    # clean tags from '>' and '<' occurences
    tags = re.sub('><', ' ', tags) 
    tags = re.sub('<|>', '', tags)
    # add single tag of tags and add it to lists and sets
    for tag in tags.split():               
        tag_list.append(tag)

#tag_set = set(tag_list)
# filter term 'machine-learning', because sql export filters for this term
tag_set = set(filter(lambda a: a != 'machine-learning', tag_list))
tag_Counter = Counter(tag_list)

print(tag_set)

{'encog', 'math', 'pybrain', 'search', 'knn', 'ensemble-learning', 'q-learning', 'euclidean-distance', 'feature-extraction', 'document-classification', 'scala', 'decision-tree', 'kaggle', 'missing-data', 'stanford-nlp', 'numpy', 'supervised-learning', 'smo', 'vectorization', 'svm', 'arff', 'perceptron', 'scipy', 'cryptography', 'metrics', 'php', 'collaborative-filtering', 'nlp', 'keyword-search', 'cascade-classifier', 'biometrics', 'dirichlet', 'datumbox', 'pam', 'apache-spark', 'twitter', 'kernel', 'compilation', 'e-commerce', 'dlib', 'oauth', 'haar-classifier', 'simplecv', 'confusion-matrix', 'ranking', 'test-data', 'object-recognition', 'dbn', 'pos-tagger', 'entity-framework', 'cuda', 'data-visualization', 'r-caret', 'arrays', 'bayesian-networks', 'cluster-analysis', 'matlab', 'neural-network', 'vgg-net', 'matplotlib', 'azure-machine-learning-studio', 'rbm', 'security', 'image-processing', 'node.js', 'tokenize', 'object-detection', 'io', 'java', 'python', 'genetic-programming', 'spa

In [4]:
# clean posts and match words
nlp = spacy.load("en_core_web_lg")
matcher = Matcher(nlp.vocab)

technology_pattern1 = [{'POS': 'PROPN', 'OP': '+'},
                       {'POS': 'NUM', 'OP': '?'}
                      ]

technology_pattern2 = [{'OP': '+', 'POS': 'PROPN'},
                       {'TEXT': '-', 'OP': '+'},
                       {'POS': 'VERB', 'OP': '+'}
                      ]

technology_pattern3 = [{'OP': '+', 'POS': 'NOUN'},
                       {'TEXT': '-', 'OP': '?'},
                       {'POS': 'PROPN', 'OP': '+'}
                      ]

ml_pattern1 = [{'LOWER': 'machine', 'OP': '!'},
                       #{'TEXT': '-', 'OP': '!'},
                       {'LOWER': 'learning', 'OP': '!'}
                      ]


word_set = set()
regex_pattern = '(<(pre|code|blockquote|a|strike)(.|\n)*?\/(pre|code|blockquote|a|strike)>)*?|<(p|b|br|br(.|\n)*?\/|sub|sup|em|strong|hr|s|i|ol|ul|li|code)*?>|<\/(p|b|br|sub|sup|em|strong|s|i|ol|ul|li|div|pre|blockquote|a|code)>|<h(.|\n)*?>(.|\n)*?<\/h(.|\n)*?>*?|(<(img|div|ol|ul|li)(.|\n)*?\/*?>)|\n'
matcher.add("match_technology1", [technology_pattern1])
matcher.add("match_technology2", [technology_pattern2])
matcher.add("match_technology3", [technology_pattern3])
#matcher.add("unmatch_ml_pattern", [ml_pattern1])

for text in stack_posts["AnswerBody"][500:2000]:
    text = re.sub(regex_pattern, '', text, flags=re.I)
    text = re.sub('\(|\)', ' ', text, flags=re.I)    
    doc = nlp(text)    
    
    matches = matcher(doc)
        
    for match_id, start, end in matches:
        word_set.add(doc[start:end])       
print("finished")

finished


In [14]:
# Term similarity

# loop through relavant words and get their vectors
technology_list = []
technology_string_list = []
technology_counter = Counter()
for tag in tag_set:
    term_vector = []    
    tag_doc = nlp(str(tag))
    #tag_vector = tag_doc.vector    
    
    for i,span in enumerate(word_set):        
        if tag_doc.similarity(span) >= 0.8:
        # very slow
        # if cosine_similarity([tag_vector], [span_vector])[0][0] >= 0.7:
            if span.text not in technology_string_list:
                technology_list.append(span)
                technology_string_list.append(span.text)
            technology_counter[span.text] = technology_counter[span.text] + 1





print("finished")

<ipython-input-14-3141c5bde907>:13: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if tag_doc.similarity(span) >= 0.8:


finished


In [ ]:
for technology in technology_list:
    print(technology)
    with open('technology_list.txt', 'a', newline='') as myfile:
        wr = csv.writer(myfile)
        wr.writerow([technology])